In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='chat_history', # template 안에 memory가 history를 저장하도록 한 곳
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"), # 누가 보냈는 지 알 수 없는, 예측하기 어려운 메세지를 가질 수 있음 (system, human, ai). Memory class로 대체될 것임
    ("human", "{question}")
])



chain = prompt | llm

chain.invoke({
    "chat_history": memory.load_memory_variables({})['chat_history'],
    "question":"My name is Nico"
})

AIMessage(content='Hello Nico! How can I assist you today?')

In [25]:
chain.predict(question="I live is Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live is Seoul

> Finished chain.


"That's great, Nico! Seoul is a vibrant and bustling city with a rich history and culture. How can I help you with anything related to Seoul?"

In [26]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live is Seoul
AI: That's great, Nico! Seoul is a vibrant and bustling city with a rich history and culture. How can I help you with anything related to Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

In [27]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Nico'),
  AIMessage(content='Hello Nico! How can I assist you today?'),
  HumanMessage(content='I live is Seoul'),
  AIMessage(content="That's great, Nico! Seoul is a vibrant and bustling city with a rich history and culture. How can I help you with anything related to Seoul?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Nico.')]}

In [17]:
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})